<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YoloV3_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```




Put this in the console , ctrl+shift+i

In [ ]:
import os
import cv2
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

## 1.Clone and build Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 15490 (delta 10), reused 23 (delta 9), pack-reused 15460
Receiving objects: 100% (15490/15490), 14.10 MiB | 21.91 MiB/s, done.
Resolving deltas: 100% (10395/10395), done.


In [ ]:
# change makefile to have gpu and opencv enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

[Errno 20] Not a directory: 'darknet'
/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
#Build(make) Darknet
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# Yolo with COCO Dataset

## 2. Yolov3 Weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## 3.Predict
    !./darknet detect <path to config> <path to weights> <path to image>

    imShow('predictions.jpg')



In [ ]:
!wget https://images.unsplash.com/photo-1583430999116-511887f6ee69?ixlib=rb-1.2.1&dl=donny-jiang-OWrC7nDbeM0-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb
!wget https://images.unsplash.com/photo-1483526682683-7560afbb5299?ixlib=rb-1.2.1&dl=elias-ehmann-eP93fjCLv4g-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb
!wget https://images.unsplash.com/photo-1520011554120-17c4c63fc213?ixlib=rb-1.2.1&dl=joseph-cooper-lEwc9W5eLH0-unsplash.jpg&q=80&fm=jpg&crop=entropy&cs=tinysrgb

In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg

In [ ]:
imShow('predictions.jpg')

In [ ]:
# LOCAL MACHINE DOWNLOAD
download('predictions.jpg')

# GOOGLE DRIVE DOWNLOAD
!cp predictions.jpg /content/drive/MyDrive/Classroom/detection1.jpg

# Yolo with Custom Dataset

In [ ]:
!git clone https://github.com/mralamdari/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 452 (delta 2), reused 0 (delta 0), pack-reused 444
Receiving objects: 100% (452/452), 34.09 MiB | 39.54 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [ ]:
%cd OIDv4_ToolKit
!pip3 install -r requirements.txt

/content/darknet/OIDv4_ToolKit


In [ ]:
!python3 main.py downloader --classes Human_eye Human_mouth --type_csv train --limit 167 --multiclasses 1
# !python3 main.py downloader --classes Human_eye Human_nose Human_hair Human_mouth Human_ear Human_beard Human_face --type_csv train --limit 1000 --multiclasses 1
# !python3 main.py downloader --classes Human_eye Human_nose Human_hair Human_mouth Human_ear Human_beard Human_face --type_csv train --limit 5000 --multiclasses 1


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading ['Human eye', 'Human mouth'] together.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 47865 KB/s, 0 sec

In [ ]:
!python3 convert_annotations.py

Currently in subdirectory: train
Converting annotations for class: Human eye_Human mouth
100% 227/227 [00:09<00:00, 23.35it/s]


In [ ]:
%cd ..

/content/darknet


#### 1.delete the old labels
#### 2.move pictures and it's new labels to darknet/data/obj folder

In [ ]:
class_names = os.listdir('/content/darknet/OIDv4_ToolKit/OID/Dataset/train')[0]
path = f'/content/darknet/OIDv4_ToolKit/OID/Dataset/train/{class_names}'
shutil.rmtree(f'{path}/Label')
os.rename(path, '/content/darknet/data/obj')

In [ ]:
with open('/content/darknet/OIDv4_ToolKit/classes.txt', 'w+') as classes_file:
  for i, class_name in enumerate(class_names.split('_')):
    classes_file.write(class_name)
    if i != len(class_names)-1:
      classes_file.write('\n')

In [ ]:
with open('/content/darknet/cfg/yolov3.cfg', 'r+') as f:
    config_files = f.readlines()

config_files[2] = '# '+config_files[2] # batch=1
config_files[3] = '# '+config_files[3] # subdivisions=1

config_files[5] = config_files[5][2:]  # batch=64
config_files[6] = config_files[6][2:]  # subdivisions=16


num_classes=3
max_batches = num_classes*2000
config_files[19] = f'max_batches={max_batches}\n'
config_files[21] = f'steps={max_batches*8//10}, {max_batches*9//10}\n'

In [ ]:
for i, j in enumerate(config_files):
    if j == '[yolo]\n':
        config_files[i-4] = f"filters={3*(5+num_classes)}\n" # convolutional filters
        config_files[i+3] = f"classes={num_classes}\n"       # yolo_classes
        config_files[i+8] = 'random=1\n'                     # yolo_random  0 or 1

In [ ]:
with open('/content/darknet/cfg/yolov3-custom.cfg', 'w+') as f:
    f.writelines(config_files)

In [ ]:
#Copy the classes name to obj.names
shutil.copy('/content/darknet/OIDv4_ToolKit/classes.txt', '/content/darknet/data/obj.names')

'/content/darknet/data/obj.names'

In [34]:
#Obj details
with open('/content/darknet/data/obj.data', 'w+') as obj_data:
  odata = f"""classes={len(class_names)}
  train=/content/darknet/data/train.txt
  valid=/content/darknet/data/test.txt
  names=/content/darknet/data/obj.names
  backup=/content/drive/MyDrive/yolo_backup
  """
  obj_data.write(odata)

In [35]:
os.makedirs('/content/drive/MyDrive/yolo_backup', exist_ok=True)

In [ ]:
# write each image's location in train.txt
images_path = '/content/darknet/data/'
with open(f"{images_path}train.txt", 'w+') as train_file:
  for p in os.listdir(f'{images_path}obj'):
    if p.endswith('.jpg'):
      train_file.write('/content/darknet/data/obj/'+p+'\n')
  train_file.close()

#### download the pre trained weights of yolo

In [ ]:
#### download the pre trained weights of yolo
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-28 07:56:01--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  59.8MB/s    in 2.6s    

2022-08-28 07:56:04 (59.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
os.rename('/content/darknet/darknet53.conv.74', '/content/darknet/darknet53_conv_74')

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-custom.cfg darknet53_conv_74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 993.692017, iou_loss = 0.000000, total_loss = 993.692017 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 5081.813965, iou_loss = 0.000000, total_loss = 5081.813965 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 239.179321, iou_loss = 0.000000, total_loss = 239.179321 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 993.283325, iou_loss = 0.000000, total_loss = 993.283325 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 5100.521973, iou_loss = 0.000000, total_loss = 5100.521973 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.0